# Investigating links between crime and grocery prices
#### Crime data from https://data.police.uk/data/archive/ accessed 19/11/2023
#### Grocery data from https://www.kaggle.com/datasets/ziedzen/uk-grocery-retailer-sales-and-pricing-analysis accessed 19/11/2023

Imports

In [ ]:
import pandas as pd
import os
import datetime
import calendar
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

Loading grocery data

In [ ]:
df_prices = pd.read_csv('PriceAdjustments.csv')
df_items = pd.read_csv('Items.csv')
df_sales = pd.read_csv('SampleSales.csv')

In [ ]:
df_prices.head()

In [ ]:
df_items.head()

In [ ]:
df_sales.head()

Loop through each local police file, compile for the appropriate time period.

In [ ]:
cwd = os.getcwd()
#make three arrays for three types of crime reports
outcomes = []
street = []
stop = [] #stop and search data
year_list = ['2018', '2019']
long_ver = False
#long_ver = True      #uncomment this for long version*
if long_ver == True:
    year_list = ['2017', '2018', '2019']
#loop through each required date
for year in year_list:
    for month in ['01','02','03','04','05','05','06'
                  ,'07','08','09','10','11','12']:
        #conditional break statement to prevent exiting the scope of the data
        if (year == '2019') and (month == '04'):
            break
        #finally get full path for each date
        rel = f'2019-07/{year}-{month}/'
        path = os.path.join(cwd, rel)
        items = os.listdir(path)
        #for each date, loop through individual local reports
        for item in items:
            name = item[:-4]
            print(path+item)
            #create a new data frame for each indvidiual report
            globals()[f'df{name}'] = pd.read_csv(path+item)
            #append each of the three categories of report
            if name[-1] == 's':
                outcomes.append(globals()[f'df{name}'])
            elif name[-1] == 't':
                street.append(globals()[f'df{name}'])
            else:
                globals()[f'df{name}']['Month'] = \
                globals()[f'df{name}']['Date'].str.slice(0, 10)
                del globals()[f'df{name}']['Date']
                stop.append(globals()[f'df{name}'])

\* long version includes dates outside of the grocery data scope. Useful for focusing on crime analysis only

In [ ]:
#end with three arrays of dataframes
outcomes[:10]

In [ ]:
street[:10]

In [ ]:
stop[:10]

In [ ]:
df_outcomes = pd.concat(outcomes)
df_street = pd.concat(street)
df_stop = pd.concat(stop)
#change to datetime
df_outcomes['Month']= pd.to_datetime(df_outcomes['Month'])
df_street['Month']= pd.to_datetime(df_street['Month'])
df_stop['Month']= pd.to_datetime(df_stop['Month'])

In [ ]:
df_outcomes.dtypes

In [ ]:
df_street.dtypes

In [ ]:
df_stop.dtypes

Grocery prices are listed per week. Create a function to take year and week number and return the month the week lands in.
Note that the first day of each week determines its year and month.

In [ ]:
def MonthFromWeek(year = None, week = None):
    if week not in range(1, 53):
        raise Exception('Please enter a week between 1 and 52')
    start = 1
    first = datetime.datetime(year, 1, 1)
    first = first.weekday()
    #start week one a few days in advance unless the year starts on a Monday
    if first !=0:
        start += 7-first
    days = 365
    #add a day for leap years
    leap = int(calendar.isleap(year))
    days += leap
    #find days into the year the given week commences
    week_com = start + (week-1)*7
    #month breaks array will have the number of the first day of each month
    month_breaks = [1]
    for i in range(1, 12):
        if i == 2:
            count = 28 + leap
        elif i in [1, 3, 5, 7, 8, 10, 12]:
            count = 31
        else:
            count = 30
        month_breaks.append(month_breaks[i-1]+count)
    if week_com == 1:
        month_out = 1
    elif week_com >= month_breaks[-1]:
        month_out = 12
    #find the first month break larger than given day and return corresponding month
    else:
        for i in range(len(month_breaks)):
            if week_com < month_breaks[i]:
                month_out = i
                break
    return month_out

In [ ]:
MonthFromWeek(2018, 33)

Some metadata

In [ ]:
print('df_prices columns\n',df_prices.columns,'\n------------------------------------\n')
print('df_items columns\n',df_items.columns,'\n------------------------------------\n')
print('df_sales columns\n',df_sales.columns,'\n------------------------------------\n')
print('df_outcomes columns\n',df_outcomes.columns,'\n------------------------------------\n')
print('df_street columns\n',df_street.columns,'\n------------------------------------\n')
print('df_stop columns\n',df_stop.columns)

Complementary function which calls MonthFromWeek() to create a datetime object.

In [ ]:
def toDate(year, week):
    month = MonthFromWeek(year, week)
    text = str(year)+'-'+str(month)
    date = pd.to_datetime(text)
    return date

In [ ]:
toDate(2018, 33)

Applying the toDate function to the prices and sales dataframes.

In [ ]:
for i in ['prices', 'sales']:
    globals()[f'df_{i}']['Month'] =\
    globals()[f'df_{i}'].apply(
            lambda x:toDate(x['year'], x['WeekIdentifier'])
            , axis = 1)

Some data exploration

In [ ]:
prices_stats = df_prices.describe()
items_stats = df_items.describe()
sales_stats = df_sales.describe()
outcomes_stats = df_outcomes.describe()
street_stats = df_street.describe()
stop_stats = df_stop.describe()

Plot for crime only

In [ ]:
#%%   #SL Only
#sl_by_month = df_street[df_street['Crime type'] == 'Shoplifting']['Month'].value_counts(sort = False)
#sl_max_month = sl_by_month.max()
#sl_min_month = sl_by_month.min()
#std_sl = (sl_by_month-sl_min_month)/(sl_max_month-sl_min_month)
#sns.lineplot(std_sl, label = 'Counts of Shoplifting'),
#plt.legend(loc = 'upper right')
#%%

In [ ]:
sns.lineplot(df_prices.groupby('Month')['BasePrice'].mean())
plt.title('Mean Price')
plt.show()
mean_price = df_prices['BasePrice'].mean()
df_street['Crime type'].value_counts()

Scaling function (min-max scaling)

In [ ]:
def minmax_scale(data):
    return (data-data.min())/(data.max()-data.min())

Generating plots

In [ ]:
sl_by_month = df_street[df_street['Crime type'] == 'Shoplifting']['Month'].value_counts(sort = False)
std_sl = minmax_scale(sl_by_month)
meanprice_by_month = df_prices.groupby('Month')['BasePrice'].mean()
std_meanprice = minmax_scale(meanprice_by_month)
sns.lineplot(std_sl, label = 'Counts of Shoplifting')
sns.lineplot(std_meanprice, label = 'Mean price')
plt.title('Standardised Price and Shoplifting')
plt.legend()
plt.show()

In [ ]:
std_meanprice_lag = pd.Series([np.nan] + std_meanprice.to_list()[:-1], std_meanprice.index.to_list())
sns.lineplot(std_sl, label = 'Counts of Shoplifting')
sns.lineplot(std_meanprice_lag, label = 'Mean price shifted by 1 Month')
plt.title('Standardised Price and Shoplifting (with time lag)')
plt.legend()
plt.show()

In [ ]:
sns.lineplot(std_sl[5:], label = 'Count of Shoplifting')
sns.lineplot(std_meanprice_lag[5:], label = 'Mean price shifted by 1 Month')
plt.title('Standardised Price and Shoplifting (with time lag)')
plt.legend(loc = 'upper right', fontsize = 9)
plt.show()

In [ ]:
sns.pointplot(x = [np.round(v, 3) for v in std_sl[5:].to_list()], y = std_meanprice_lag[5:].to_list())
plt.xlabel('Count of Shoplifting')
plt.ylabel('Mean price shifted by 1 Month')
plt.title('Standardised Price against Shoplifting (with time lag)')
plt.show()

In [ ]:
crime_by_month = df_street.groupby('Month').size()
std_crime = minmax_scale(crime_by_month)
sns.lineplot(std_meanprice)
sns.lineplot(std_crime)

Defining a function to perform a permutation test of spearman rank

In [ ]:
x_sl = std_sl.to_list()[5:]
x_crime = std_crime.to_list()[5:]
y = std_meanprice_lag.to_list()[5:]
def statistic(stat):
    rs = stats.spearmanr(stat, y).statistic
    return rs
def inverse_statistic(stat):
    rs = stats.spearmanr(stat, x_sl).statistic
    return rs

In [ ]:
stats.permutation_test((x_sl, ), statistic, alternative = 'greater', permutation_type = 'pairings')

More Comparisons - comparisons against crime overall

In [ ]:
crime_by_month = df_street['Month'].value_counts(sort = False)
std_crime = minmax_scale(crime_by_month)
sns.lineplot(std_crime, label = 'Counts of Crime')
sns.lineplot(std_meanprice, label = 'Mean price')
plt.title('Standardised Price and Crime')
plt.legend()
plt.show()

In [ ]:
sns.pointplot(x = [np.round(v, 3) for v in std_crime[5:].to_list()], y = std_meanprice_lag[5:].to_list())
plt.xlabel('Count of Crime')
plt.ylabel('Mean price shifted by 1 Month')
plt.title('Standardised Price against Crime (with time lag)')
plt.show()

In [ ]:
stats.permutation_test((x_crime, ), statistic, alternative = 'greater', permutation_type = 'pairings')

Analysis of specific item prices against shoplifting patterns, begin with  apair of functions getCategory and getSubCategory which convert input item ID into the category or subcategory of the item.

In [ ]:
def getCategory(item_id):
    return df_items[df_items['ItemID'] == item_id]['Category'].to_list()[0]

In [ ]:
def getSubCategory(item_id):
    return df_items[df_items['ItemID'] == item_id]['SubCategory'].to_list()[0]

Example

In [ ]:
getCategory('GKuS9uU17')

In [ ]:
df_prices['Category'] = df_prices['ItemID'].apply(getCategory)
df_prices['SubCategory'] = df_prices['ItemID'].apply(getSubCategory)

In [ ]:
plots = False
#plots = True     #set plots to True to plot each item category and sub category
cat_test_dict = {}    #dictionary for perumtation test results for each item category
subcat_test_dict = {}   #as above, for each subcategory
for cat in df_prices.Category.unique():
    cat_test = df_prices[df_prices['Category'] == cat].groupby('Month')['BasePrice'].mean().sort_index()
    std_cat_test = minmax_scale(cat_test)
    x_cat_test = std_cat_test.to_list()[5:]
    if len(x_cat_test) != len(x_sl):  #remove categories of items that are not availible each month
        continue
    print('Success')
    if plots == True:
        sns.pointplot(x = [np.round(v, 3) for v in std_sl[5:].to_list()], y = x_cat_test)
        plt.xlabel('Count of Shoplifting')
        plt.ylabel(cat)
        plt.title('Standardised category price and Shoplifting')
        plt.show()
    cat_perm_test = stats.permutation_test((x_cat_test, ), inverse_statistic,
                                            alternative = 'greater', permutation_type = 'pairings')
    print(cat_perm_test)
    cat_test_dict[cat] = (cat_perm_test.statistic,
                          cat_perm_test.pvalue)
#repeat for sub-categories
for subcat in df_prices.SubCategory.unique():
    subcat_test = df_prices[df_prices['Category'] == subcat].groupby('Month')['BasePrice'].mean().sort_index()
    std_subcat_test = minmax_scale(subcat_test)
    x_subcat_test = std_subcat_test.to_list()[5:]
    if len(x_subcat_test) != len(x_sl):
        continue
    print('Success')
    if plots == True:
        sns.pointplot(x = [np.round(v, 3) for v in std_sl[5:].to_list()], y = x_subcat_test)
        plt.xlabel('Count of Shoplifting')
        plt.ylabel(subcat)
        plt.title('Standardised subcategory price and Shoplifting')
        plt.show()
    subcat_perm_test = stats.permutation_test((x_subcat_test, ), inverse_statistic,
                                            alternative = 'greater', permutation_type = 'pairings')
    print(subcat_perm_test)
    subcat_test_dict[subcat] = (subcat_perm_test.statistic,
                                subcat_perm_test.pvalue)
full_test_dict = {'cat': cat_test_dict, 'subcat': subcat_test_dict}
print(cat_test_dict)
print(subcat_test_dict)
print(full_test_dict)

FileNotFoundError: ignored